#Chapter9. 데이터 수집과 그룹 연산
> 데이터 집합을 분류하고 각 그룹별로 집계나 변형 같은 어떤 함수를 적용하는건 데이터 분석과정에서 매우 중요한 일<br />
> pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공<br />
> 쿼리문은 그룹연산에 제약이 있다. 파이썬과 pandas의 강력한 표현력을 잘 이용하면 아주 복잡한 그룹연산도 pandas객체나 NumPy배열을 받는 함수의 조합을 통해 해결<br >

###이 장에서 배우게 될 내용
* 하나 이상의 키(함수, 배열, DataFrame의 칼럼 이름)을 이용하여 pandas객체를 여러조각으로 나누는 방법
* 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약통계를 계산하는 방법
* DataFrame의 각 칼럼에 다양한 함수를 적용하는 방법
* 정규화, 선형회귀, 등급 또는 부분집합 선택같은 집단 내 변형이나 다른 조작을 적용하는 방법
* 피벗테이블과 교차일람표를 구하는 방법
* 변위치 분석과 다른 데이터 파생 집단분석을 수행하는 방법

##9.1 GroupBy 메카닉
> ###*분리*-*적용*-*결합*
1. 데이터를 하나 이상의 색인을 기준으로 ***분리***한다.
2. 분리하고 나면 함수를 각 그룹에 ***적용***시켜 새로운 값을 얻어낸다.
3. 함수를 적용한 결과를 하나의 객체로 ***결합***한다.

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [2]:
df = DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                'key2':['one', 'two', 'one', 'two', 'one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})

In [3]:
df

,data1,data2,key1,key2
0,-1.140920,-1.507435,a,one
1,-0.609759,1.523876,a,two
2,-1.513366,0.716890,b,one
3,0.688068,-0.097003,b,two
4,-0.538877,-0.164868,a,one


In [4]:
grouped = df['data1'].groupby(df['key1']);

In [5]:
grouped #groupby객체 df['key1']로 참조되는 중간값에 대한 것 외에는 아무것도 계산되지 않은 객체

In [6]:
grouped.mean() #그룹별 평균

key1
a      -0.763185
b      -0.412649
Name: data1, dtype: float64

데이터가 그룹의 색인에 따라 수집되고 key1 칼럼에 있는 유일한 값으로 색인되는 새로운 series객체가 생성됨<br />
만약 여러개의 배열을 리스트로 넘겼다면 좀 더 다른 결과를 얻을수 있다.

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -0.839898
      two    -0.609759
b     one    -1.513366
      two     0.688068
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.839898,-0.609759
b,-1.513366,0.688068


In [10]:
help(means.unstack)

Help on method unstack in module pandas.core.series:

unstack(self, level=-1) method of pandas.core.series.Series instance
    Unstack, a.k.a. pivot, Series with MultiIndex to produce DataFrame
    
    Parameters
    ----------
    level : int, string, or list of these, default last level
        Level(s) to unstack, can pass level name
    
    Examples
    --------
    >>> s
    one  a   1.
    one  b   2.
    two  a   3.
    two  b   4.
    
    >>> s.unstack(level=-1)
         a   b
    one  1.  2.
    two  3.  4.
    
    >>> s.unstack(level=0)
       one  two
    a  1.   2.
    b  3.   4.
    
    Returns
    -------
    unstacked : DataFrame



아래 예제는 그룹의 색인 모두 Series객체인데, 길이만 같다면 어떤 배열이라도 상관없다.

In [11]:
states = np.array(['Ohio', 'Califonia', 'Califonia', 'Ohio', 'Ohio'])

In [12]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [13]:
statYear = df['data1'].groupby([states, years])

In [14]:
statYear

In [15]:
statYear.mean()

Califonia  2005   -0.609759
           2006   -1.513366
Ohio       2005   -0.226426
           2006   -0.538877
Name: data1, dtype: float64

In [16]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.763185,-0.049476
b,-0.412649,0.309943


key2의 칼럼이 빠진것을 볼수 있는데 그 이유는 key2가 숫자데이터가 아니기때문이다. 이런 칼럼은 *성가신 칼럼*이라고 부르며 결과에서 제외시킨다.

In [17]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

##9.1.1 그룹 간 순회하기
groupBy객체는 이터레이션(되풀이)을 지원, 그룹이름과 그에 따른 데이터 묶음을 튜플로 반환

In [18]:
for name, group in df.groupby('key1'):
    print name
    print group

a
      data1     data2 key1 key2
0 -1.140920 -1.507435    a  one
1 -0.609759  1.523876    a  two
4 -0.538877 -0.164868    a  one
b
      data1     data2 key1 key2
2 -1.513366  0.716890    b  one
3  0.688068 -0.097003    b  two


In [19]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0 -1.140920 -1.507435    a  one
4 -0.538877 -0.164868    a  one
a two
      data1     data2 key1 key2
1 -0.609759  1.523876    a  two
b one
      data1    data2 key1 key2
2 -1.513366  0.71689    b  one
b two
      data1     data2 key1 key2
3  0.688068 -0.097003    b  two


당연한 말이겠지만 이 안에서 원하는 데이터만 골라낼 수 있다. 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꿔서 유용하게 사용 가능

In [20]:
pieces = dict(list(df.groupby('key1')))

In [21]:
pieces

{'a':       data1     data2 key1 key2
 0 -1.140920 -1.507435    a  one
 1 -0.609759  1.523876    a  two
 4 -0.538877 -0.164868    a  one, 'b':       data1     data2 key1 key2
 2 -1.513366  0.716890    b  one
 3  0.688068 -0.097003    b  two}

In [22]:
pieces['b']

,data1,data2,key1,key2
2,-1.513366,0.716890,b,one
3,0.688068,-0.097003,b,two


groupby메서드는 기본적으로 axis = 0에 대해서 그룹을 만드는데, 다른축으로 그룹을 만드는 것도 가능

In [23]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [24]:
grouped = df.groupby(df.dtypes, axis = 1)

In [25]:
grouped

In [26]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -1.140920 -1.507435
 1 -0.609759  1.523876
 2 -1.513366  0.716890
 3  0.688068 -0.097003
 4 -0.538877 -0.164868, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

##9.1.2 칼럼 또는 칼럼의 일부만 선택하기
groupby객체를 칼럼 이름이나 칼럼 이름이 담긴 배열로 색인하면 수집을 위해 *해당칼럼*을 선택하게 된다.

In [27]:
df.groupby('key1')['data1']

In [28]:
df.groupby('key1')['data1'].mean()

key1
a      -0.763185
b      -0.412649
Name: data1, dtype: float64

In [29]:
df.groupby('key1')[['data2']]

In [30]:
df.groupby('key1')[['data2']].mean()

,data2
key1,
a,-0.049476
b,0.309943


색인으로 얻는 객체는 groupby 메서드에 리스트나 배열을 넘기면 DataFrameGroupBy객체가 되고, 단일값으로 칼럼 이름을 하나만 넘기면 SeriesGroupBy객체가 된다.

##9.1.3 사전과 Series에서 묶기
그룹 정보는 배열이 아닌 형태로 존재하기도 한다.

In [120]:
people = DataFrame(np.random.randn(5,5),
                  columns = ['a', 'b', 'c', 'd', 'e'],
                  index = ['Joe', 'Steve', 'Wes', 'Jim', 'Trevis'])

In [32]:
people

,a,b,c,d,e
Joe,-1.220578,1.438065,0.188507,0.705883,-0.369075
Steve,0.555838,0.041022,0.888348,-2.400452,0.926124
Wes,0.036805,0.529801,-0.527697,-0.671109,0.210652
Jim,-1.151381,-0.173391,1.134535,0.717705,0.634761
Trevis,-0.218087,1.076053,0.810081,-1.663764,0.429935


In [33]:
people.ix[2:3, ['b', 'c']] = np.nan

In [34]:
people

,a,b,c,d,e
Joe,-1.220578,1.438065,0.188507,0.705883,-0.369075
Steve,0.555838,0.041022,0.888348,-2.400452,0.926124
Wes,0.036805,NaN,NaN,-0.671109,0.210652
Jim,-1.151381,-0.173391,1.134535,0.717705,0.634761
Trevis,-0.218087,1.076053,0.810081,-1.663764,0.429935


In [35]:
mapping = {'a':'red', 'b':'red', 'c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

In [36]:
by_column = people.groupby(mapping, axis = 1)

In [37]:
by_column

In [38]:
by_column.sum()

,blue,red
Joe,0.894390,-0.151588
Steve,-1.512104,1.522984
Wes,-0.671109,0.247457
Jim,1.852239,-0.690011
Trevis,-0.853684,1.287901


In [39]:
from pandas import Series

In [40]:
map_series = Series(mapping)

In [41]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Trevis,2,3


##9.1.4 함수로 묶기
사전이나 Series를 사용하는 것에 비해 파이썬 함수를 사용해서 그룹을 매핑하는것은 좀 더 독창적이고 추상적인 방법.

In [43]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.335154,1.264674,1.323041,0.752479,0.476339
5,0.555838,0.041022,0.888348,-2.400452,0.926124
6,-0.218087,1.076053,0.810081,-1.663764,0.429935


내부적으로 모두 배열로 변환되므로 함수와 배열, 사전 또는 Series를 함께 섞어 쓰는것도 전혀 문제가 되지 않는다.

In [44]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [45]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.220578  1.438065  0.188507 -0.671109 -0.369075
  two -1.151381 -0.173391  1.134535  0.717705  0.634761
5 one  0.555838  0.041022  0.888348 -2.400452  0.926124
6 two -0.218087  1.076053  0.810081 -1.663764  0.429935

##9.1.5 색인 단계로 묶기
계층적으로 색인된 데이터 묶음은 축 색인의 단계 중 하나를 사용해서 편리하게 모을 수 있는 기능을 제공한다.<br />
이 기능을 사용하려면 level인자를 통해 레벨 번호나 이름을 넘기면 된다.

In [46]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'KR', 'KR'],
                                    [1, 3, 5, 1, 3]], names = ['cty', 'tenor'])

In [47]:
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)

In [48]:
hier_df

cty          US                            KR          
tenor         1         3         5         1         3
0     -0.062576  1.473723 -2.115714 -0.732727 -1.041743
1     -0.144869  1.560652  0.876048  0.021882  0.150959
2     -0.222757 -0.473839 -1.186367  0.499311  1.418829
3      0.026889  0.443408 -0.786914 -0.247936  0.301954

In [49]:
hier_df.groupby(level='cty', axis = 1).count()

cty,KR,US
0,2,3
1,2,3
2,2,3
3,2,3


In [50]:
hier_df.groupby(level='tenor', axis = 1).mean()

tenor,1,3,5
0,-0.397651,0.215990,-2.115714
1,-0.061494,0.855806,0.876048
2,0.138277,0.472495,-1.186367
3,-0.110524,0.372681,-0.786914


##9.2 데이터 수집
위 예제에서는 mean, count, sum, min등을 통해 스칼라값을 구했지만 직접 고안한 방식과 추가적으로 기존 메서드를 호출하는 방식으로 데이터를 집계할 수 있다.

In [51]:
df

,data1,data2,key1,key2
0,-1.140920,-1.507435,a,one
1,-0.609759,1.523876,a,two
2,-1.513366,0.716890,b,one
3,0.688068,-0.097003,b,two
4,-0.538877,-0.164868,a,one


In [52]:
grouped = df.groupby('key1')

In [53]:
grouped['data1']

In [54]:
grouped['data1'].quantile(0.9) #4분위수

key1
a      -0.553053
b       0.467925
Name: data1, dtype: float64

자신만의 데이터 집계함수를 사용하려면 배열의 agg나 aggregate 메서드에 해당 함수를 넘기면 된다.

In [55]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [56]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.602043,3.031311
b,2.201435,0.813893


describe같은 메서드를 사용해서 데이터를 집계할 수도 있다.

In [57]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.763185 -0.049476
     std    0.329042  1.518946
     min   -1.140920 -1.507435
     25%   -0.875340 -0.836151
     50%   -0.609759 -0.164868
     75%   -0.574318  0.679504
     max   -0.538877  1.523876
b    count  2.000000  2.000000
     mean  -0.412649  0.309943
     std    1.556649  0.575509
     min   -1.513366 -0.097003
     25%   -0.963008  0.106470
     50%   -0.412649  0.309943
     75%    0.137710  0.513417
     max    0.688068  0.716890

* 최적화 된 groupby 메서드

함수이름       |설명    
:------------|:-----
count        |Nu 그룹내에 NA값이 아닌 값의 수를 반환한다.   
sum          |NA 값이 아닌 값들의 합을 구한다.   
mean         |NA 값이 아닌 값들의 평균 값을 구한다.   
median       |NA 값이 아닌 값들의 산술 중간 값을 구한다.   
std, var     |편향되지 않은(n-1을 분모로 하는)표준편차와 분산   
min, max     |NA 값이 아닌 값 중 최소 값과 최대 값   
prod         |NA 값이 아닌 값의 곱   
first, last  |NA 값이 하닌 값들 중 첫번째 값과 마지막 값   

집계의 고급 기능을 설명하기 위해 레스토랑 팁에 대한 데이터를 사용하려고 한다.<br />
데이터를 불러온 후 팁의 비율을 담기 위한 칼럼인 tip_pct를 추가했다.

In [58]:
tips = pd.read_csv('tips.csv')

In [59]:
#전체 지불 금액대비 팁의 비율 추가
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [60]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


##9.2.1 칼럼에 여러가지 함수 적용하기

In [61]:
grouped = tips.groupby(['sex', 'smoker'])

In [62]:
grouped_pct = grouped['tip_pct']

In [63]:
grouped_pct.agg('mean') #함수 이름을 문자열로 넘겨도 된다.

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

만일 함수의 목록이나 함수 이름들을 넘기면 함수 이름을 칼럼이름으로 하는 DataFrame을 얻게 된다.

In [64]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

groupby객체에서 자동으로 지정되는 칼럼 이름을 그대로 쓰지 않아도 된다. 이름과 함수가 담긴 튜플의 리스트를 넘기면 각 튜플에서 첫번째 원소는 DataFrame에서 칼럼의 이름으로 사용된다.

In [65]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std), ('zoo', peak_to_peak)])

foo       bar       zoo
sex    smoker                              
Female No      0.156921  0.036421  0.195876
       Yes     0.182150  0.071595  0.360233
Male   No      0.160669  0.041849  0.220186
       Yes     0.152771  0.090588  0.674707

In [66]:
functions = ['count', 'mean', 'max']

In [67]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [68]:
result

tip_pct                      total_bill                  
                 count      mean       max       count       mean    max
sex    smoker                                                           
Female No           54  0.156921  0.252672          54  18.105185  35.83
       Yes          33  0.182150  0.416667          33  17.977879  44.30
Male   No           97  0.160669  0.291990          97  19.791237  48.33
       Yes          60  0.152771  0.710345          60  22.284500  50.81

In [69]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [70]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [71]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct                total_bill            
               Durchschnitt  Abweichung  Durchschnitt  Abweichung
sex    smoker                                                    
Female No          0.156921    0.001327     18.105185   53.092422
       Yes         0.182150    0.005126     17.977879   84.451517
Male   No          0.160669    0.001751     19.791237   76.152961
       Yes         0.152771    0.008206     22.284500   98.244673

칼럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 칼럼 이름에 대응하는 함수가 들어있는 사전을 넘기면 된다.

In [72]:
grouped.agg({'tip':np.max, 'size':'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [73]:
grouped.agg({'tip_pct':['min', 'max', 'mean', 'std'], 'size':'sum'})

tip_pct                                size
                    min       max      mean       std   sum
sex    smoker                                              
Female No      0.056797  0.252672  0.156921  0.036421   140
       Yes     0.056433  0.416667  0.182150  0.071595    74
Male   No      0.071804  0.291990  0.160669  0.041849   263
       Yes     0.035638  0.710345  0.152771  0.090588   150

##9.2.2 색인되지 않은 형태로 집계된 데이터 반환하기
지금까지 살펴본 모든 예제에서 집계된 데이터는 유일한 그룹키 조합으로 색인되어 반환되었다. 하지만 항상 이런 동작을 기대하는 것은 아니므로 groupby메서드에 as_index=False를 념겨서 색인되지 않도록 할수 있다.

In [74]:
tips.groupby(['sex', 'smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [75]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


reset_index 메서드를 호출해 같은 결과를 얻을수 있다.

In [76]:
tips.groupby(['sex', 'smoker']).mean().reset_index()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


#9.3 그룹별 연산과 변형
다양한 그룹 연산을 수행할 수 있는 transform과 apply 메서드에 대해서 알아보도록 하자.

In [77]:
df

,data1,data2,key1,key2
0,-1.140920,-1.507435,a,one
1,-0.609759,1.523876,a,two
2,-1.513366,0.716890,b,one
3,0.688068,-0.097003,b,two
4,-0.538877,-0.164868,a,one


In [78]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')

In [79]:
k1_means

,mean_data1,mean_data2
key1,,
a,-0.763185,-0.049476
b,-0.412649,0.309943


In [80]:
help(pd.merge) #데이터베이스 스타일이 열 또는 인덱스에 의해 결합 작업을 수행하여 데이터 프레임 개체를 병합합니다.

Help on function merge in module pandas.tools.merge:

merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True)
    Merge DataFrame objects by performing a database-style join operation by
    columns or indexes.
    
    If joining columns on columns, the DataFrame indexes *will be
    ignored*. Otherwise if joining indexes on indexes or indexes on a column or
    columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        * left: use only keys from left frame (SQL: left outer join)
        * right: use only keys from right frame (SQL: right outer join)
        * outer: use union of keys from both frames (SQL: full outer join)
        * inner: use intersection of keys from both frames (SQL: inner join)
    on : label or list
        Field names to join on. Must be fo

In [85]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-1.140920,-1.507435,a,one,-0.763185,-0.049476
1,-0.609759,1.523876,a,two,-0.763185,-0.049476
4,-0.538877,-0.164868,a,one,-0.763185,-0.049476
2,-1.513366,0.716890,b,one,-0.412649,0.309943
3,0.688068,-0.097003,b,two,-0.412649,0.309943


원하는 대로 동작하긴 하지만 뭔가 매끄럽지 못하다. 두 데이터의 칼럼을 np.mean함수를 사용해서 변형하는 작업이라고 생각할 수 있다.

In [86]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.763185,-0.049476
b,-0.412649,0.309943


In [87]:
df.groupby('key1').transform(np.mean)

,data1,data2
0,-0.763185,-0.049476
1,-0.763185,-0.049476
2,-0.412649,0.309943
3,-0.412649,0.309943
4,-0.763185,-0.049476


앞부분에서 살펴봤던 people DataFrame을 떠올려보자. 그리고 groupBy객체에 대해 transform메서드를 호출해보자

In [104]:
help(df.groupby('key1').transform)
# 각 그룹의 같은 인덱스 데이터 프레임을 생성하고 변환 된 값으로 채워진 원본과 같은 인덱스를 가지는 데이터 프레임을 반환합니다

Help on method transform in module pandas.core.groupby:

transform(self, func, *args, **kwargs) method of pandas.core.groupby.DataFrameGroupBy instance
    Call function producing a like-indexed DataFrame on each group and
    return a DataFrame having the same indexes as the original object
    filled with the transformed values
    
    Parameters
    ----------
    f : function
        Function to apply to each subframe
    
    Notes
    -----
    Each subframe is endowed the attribute 'name' in case you need to know
    which group you are working on.
    
    Examples
    --------
    >>> grouped = df.groupby(lambda x: mapping[x])
    >>> grouped.transform(lambda x: (x - x.mean()) / x.std())



In [127]:
people

,a,b,c,d,e
Joe,-0.755395,-1.440310,-1.455435,0.353489,-0.472265
Steve,0.633246,-1.856209,-0.541853,0.585649,-0.215505
Wes,1.443545,-1.707569,-0.148126,-0.366871,-0.142898
Jim,-0.046594,1.468637,0.687312,0.699155,-0.296441
Trevis,-0.505695,-0.440826,-0.505743,1.105472,-1.057563


In [147]:
key = ['one', 'two', 'one', 'two', 'one']

In [148]:
people.groupby(key).mean()

,a,b,c,d,e
one,0.060818,-1.196235,-0.703102,0.364030,-0.557575
two,0.293326,-0.193786,0.072729,0.642402,-0.255973


In [149]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Jim,NaN,NaN,NaN,NaN,NaN
Joe,NaN,NaN,NaN,NaN,NaN
Steve,NaN,NaN,NaN,NaN,NaN
Trevis,NaN,NaN,NaN,NaN,NaN
Wes,NaN,NaN,NaN,NaN,NaN
one,0.060818,-1.196235,-0.703102,0.364030,-0.557575
two,0.293326,-0.193786,0.072729,0.642402,-0.255973


책에는 잘 들어가 있는데..제대로 실행이 안됀거 같습니다. 왜 그런지 다같이 한번 생각해보면 좋을거 같아요 ㅎㅎㅎ;;;

In [150]:
for name, group in people.groupby(key):
    print name
    print group

one
               a         b         c         d         e
Joe    -0.755395 -1.440310 -1.455435  0.353489 -0.472265
Wes     1.443545 -1.707569 -0.148126 -0.366871 -0.142898
Trevis -0.505695 -0.440826 -0.505743  1.105472 -1.057563
two
              a         b         c         d         e
Steve  0.633246 -1.856209 -0.541853  0.585649 -0.215505
Jim   -0.046594  1.468637  0.687312  0.699155 -0.296441


In [151]:
def demean(arr):
    return arr - arr.mean()

In [152]:
demeaned = people.groupby(key).transform(demean)

In [153]:
demeaned

,a,b,c,d,e
Jim,-0.339920,1.662423,0.614582,0.056753,-0.040468
Joe,-0.816213,-0.244075,-0.752334,-0.010541,0.085311
Steve,0.339920,-1.662423,-0.614582,-0.056753,0.040468
Trevis,-0.566514,0.755409,0.197359,0.741442,-0.499988
Wes,1.382727,-0.511334,0.554975,-0.730901,0.414678


In [155]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,0.460909,-0.170445,0.184992,-0.243634,0.138226
two,-0.691363,0.255667,-0.277488,0.365450,-0.207339


이것 역시 다른결과가 출력이 되는군요.

##9.3.1 apply: 분리-적용-병합
transform은 엄격한 요구사항을 갖는 특수한 목적의 함수다. 인자로 넘긴 함수는 반드시 스칼라 값을 생성해야 하며 혹은 같은 크기를 가지는 변형된 배열을 생성해야 한다. 가장 일반적인 groupby 메서드의 목적은 apply인데, 이는 지금부터 다룰 내용이다.

앞에서 살펴보았던 팁 데이터에서 그룹별 상위 5개의 tip_pct 값을 골라보자. 우선 특정 갈럼에서 가장 큰 값을 가지는 로우를 선택하는 함수를 바로 작성해보자.

In [156]:
def top(df, n=5, column='tip_pct'):
    return df.sort_index(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


이제 흡연자(smoker) 그룹에 대하여 이 함수를 apply하게 되면 다음과 같은 결과를 얻을 수 있다.

In [157]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

이 결과를 보면 top함수가 나누어진 DataFrame의 각 부분에 모두 적용이 되었고, pandas, concat을 이용해서 하나로 합쳐진 다음 그룹 이름표가 붙여졌다. 결과는 계층적 색인을 가지게 되고 내부 색인은 원본 DataFrame의 색인 값을 가지게 된다.

만일 apply 메서드에 넘길 함수가 추가적인 인자를 받는다면 함수 이름뒤에 붙여서 넘겨주면 된다.

In [158]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [159]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [160]:
result

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
dtype: float64

In [163]:
result.unstack('smoker')

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


describe 같은 메서드를 호출하면 GroupBy 내부적으로 다음과 같은 단계를 수행한다.
```python
f = lambda x: x.describe()
grouped.apply(x)
```

### 그룹 색인 생략하기

앞에서 살펴 본 예제에서 반환된 객체들은 원본 객체의 각 조각에 대한 색인과 그룹키가 계층적 색인으로 사용되는 걸 볼 수 있다. 이런 결과는 groupby 메서드에 group_keys=False를 넘겨서 막을수 있다.

In [164]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [165]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


## 9.3.2 변위치 분석과 버킷 분석
7장에서 살펴본 내용을 떠올려보면 pandas의 cut, qcut이라는 메서드를 사용해서 선택한 크기만큼 혹은 표본 변위치에 따라 데이터를 나눌 수 있었다. 이 함수를 groupby와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 아주 쉽게 수행할 수 있다.

In [166]:
frame = DataFrame({'data1':np.random.randn(1000),
                  'data2':np.random.randn(1000)})

In [167]:
frame

,data1,data2
0,0.224421,0.746222
1,0.387424,0.768549
2,0.753908,-0.608434
3,-0.998499,-0.198726
4,0.684612,0.350964
5,0.705289,0.122795
6,0.356493,1.423921
7,1.727190,0.069649
8,-0.475763,0.624772
9,-0.534546,-0.042484


In [174]:
factor = pd.cut(frame.data1, 4)
factor[:10]

  (-0.537, 1.313]
  (-0.537, 1.313]
  (-0.537, 1.313]
 (-2.388, -0.537]
  (-0.537, 1.313]
  (-0.537, 1.313]
  (-0.537, 1.313]
   (1.313, 3.164]
  (-0.537, 1.313]
  (-0.537, 1.313]
Levels (4): Index(['(-4.246, -2.388]', '(-2.388, -0.537]',
                   '(-0.537, 1.313]', '(1.313, 3.164]'], dtype=object)

cut 메서드는 Factor 객체를 반환하는데, 이 Factor객체는 바로 groupby로 넘길 수 있다.

In [170]:
def get_stats(group):
    return {'min':group.min(), 'max':group.max(), 'count':group.count(), 'mean':group.mean()}

In [171]:
grouped = frame.data2.groupby(factor)

In [172]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-4.246, -2.388]",8,1.131999,-0.215569,-1.894985
"(-2.388, -0.537]",299,2.009005,-0.103146,-2.704931
"(-0.537, 1.313]",604,3.115139,0.005312,-3.269123
"(1.313, 3.164]",89,3.810381,0.112956,-2.197447


이는 등간격 버킷이고, 표본 변위치에 기반하여 크기가 같은 버킷을 계산하려면 qcut을 사용해야 한다.

In [183]:
#변위치 숫자를 반환한다.
grouping = pd.qcut(frame.data1, 10, labels=False)
grouping

array([5, 6, 7, 1, 7, 7, 6, 9, 3, 3, 8, 2, 3, 0, 1, 5, 1, 5, 4, 4, 8, 8, 1,
       1, 9, 3, 0, 5, 6, 0, 7, 3, 0, 0, 9, 9, 5, 3, 0, 0, 9, 5, 7, 1, 2, 0,
       0, 5, 0, 0, 2, 4, 4, 1, 3, 2, 7, 3, 6, 9, 5, 6, 6, 4, 5, 6, 2, 8, 1,
       9, 7, 4, 1, 1, 1, 2, 2, 9, 2, 4, 4, 0, 4, 9, 8, 0, 2, 3, 3, 8, 8, 7,
       3, 7, 6, 8, 6, 7, 6, 1, 7, 0, 1, 5, 9, 1, 2, 6, 7, 7, 0, 1, 1, 6, 1,
       4, 3, 4, 0, 8, 2, 7, 7, 6, 6, 9, 3, 0, 6, 9, 1, 3, 8, 8, 7, 8, 7, 9,
       9, 0, 6, 1, 3, 2, 6, 2, 7, 5, 5, 9, 9, 0, 1, 2, 3, 7, 0, 6, 0, 0, 6,
       6, 2, 5, 4, 8, 4, 6, 2, 1, 2, 8, 2, 4, 2, 6, 7, 7, 6, 2, 7, 5, 4, 2,
       2, 0, 5, 6, 5, 2, 5, 6, 5, 4, 3, 6, 4, 7, 8, 4, 7, 6, 3, 9, 5, 6, 7,
       1, 8, 2, 2, 8, 2, 9, 2, 6, 0, 3, 0, 3, 5, 7, 6, 1, 9, 8, 0, 5, 9, 7,
       8, 1, 1, 0, 6, 8, 4, 4, 6, 1, 8, 8, 6, 8, 7, 2, 0, 2, 7, 7, 5, 2, 7,
       4, 4, 2, 2, 2, 9, 9, 8, 3, 3, 1, 0, 2, 8, 3, 2, 5, 2, 6, 9, 9, 5, 5,
       1, 0, 8, 9, 5, 2, 2, 6, 1, 3, 3, 2, 2, 7, 4, 4, 6, 8, 3, 0, 4, 4, 3,
       2, 7,

In [184]:
grouped = frame.data2.groupby(grouping)

In [185]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
0,100,1.832354,-0.040376,-2.685284
1,100,1.939753,-0.172506,-2.615642
2,100,2.009005,-0.113665,-2.704931
3,100,2.420816,-0.032912,-3.269123
4,100,1.739162,-0.176065,-2.100690
5,100,2.423690,-0.155260,-3.031191
6,100,2.888340,0.269432,-2.151635
7,100,3.115139,0.055077,-2.247539
8,100,2.470646,0.082852,-2.240582
9,100,3.810381,0.090387,-2.197447


##9.3.3 예제 : 그룹의 국한된 값으로 누락된 값 채우기
누락된 데이터를 정리할때면 dropna를 사용해서 데이터를 걸러내는 경우가 있는 반면, 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 어떤 값으로 채우고 싶을때도 있다. 이런 경우에 fillna메서드를 사용하는데, 다음은 누락된 값을 평균값으로 대체하는 예제다

In [186]:
s = Series(np.random.randn(6))

In [187]:
s[::2] = np.nan

In [188]:
s

0         NaN
1   -0.828925
2         NaN
3   -1.157968
4         NaN
5   -0.327647
dtype: float64

In [189]:
s.fillna(s.mean())

0   -0.771513
1   -0.828925
2   -0.771513
3   -1.157968
4   -0.771513
5   -0.327647
dtype: float64

그룹별로 채워넣고 싶은 값이 다르다고 하자. 데이터를 그룹으로 나누고 apply 함수를 사용해서 각 그룹에 대해 fillna를 적용하면 된다.

In [192]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'Califonia', 'Idaho']

In [191]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [193]:
data = Series(np.random.randn(8), index = states)

In [194]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [195]:
data

Ohio         0.255182
New York     0.411025
Vermont           NaN
Florida      0.310343
Oregon       0.828543
Nevada            NaN
Califonia    0.845606
Idaho             NaN
dtype: float64

In [196]:
data.groupby(group_key).mean()

East    0.325516
West    0.837075
dtype: float64

In [199]:
fill_mean = lambda g: g.fillna(g.mean())

In [200]:
data.groupby(group_key).apply(fill_mean)

Ohio         0.255182
New York     0.411025
Vermont      0.325516
Florida      0.310343
Oregon       0.828543
Nevada       0.837075
Califonia    0.845606
Idaho        0.837075
dtype: float64

아니면 그룹에 따라 미리 정의된 다른 값을 채워넣어야 할 경우도 있다. 각 그룹은 내부적으로 name이라는 속성을 가지고 있으므로 이를 이용하자.

In [201]:
fill_values = {'East':0.5, 'West':-1}

In [204]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [205]:
data.groupby(group_key).apply(fill_func)

Ohio         0.255182
New York     0.411025
Vermont      0.500000
Florida      0.310343
Oregon       0.828543
Nevada      -1.000000
Califonia    0.845606
Idaho       -1.000000
dtype: float64